# **CV를 이용한 차선 인식 실습**

드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

드라이브에 경로로 이동. 파일 또는 폴더 우측에 "..." 표시를 눌러서 "경로 복사"

In [ ]:
%cd /content/drive/MyDrive/colab

/content/drive/.shortcut-targets-by-id/112/colab


tensorflow version downgrade를 위해 2.4.0 삭제

In [ ]:
%pip uninstall tensorflow

Uninstalling tensorflow-2.4.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.0


tesnsorflow 1.15.0 설치

In [ ]:
%pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 31kB/s 
     |████████████████████████████████| 512kB 36.1MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 3.8MB 43.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=47c4339beb8ea2639d01f1aad8f18bc935fb993c75f64af1a5d0d056e7075c46
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninsta

In [ ]:
%pip install gast==0.3.2

  Created wheel for gast: filename=gast-0.3.2-cp36-none-any.whl size=9680 sha256=b7346e064d000c7fc44cf977fffc691d63cdb962de3d3a3d1eef3eec87062214
  Stored in directory: /root/.cache/pip/wheels/59/38/c6/234dc39b4f6951a0768fbc02d5b7207137a5b1d9094f0d54bf
Successfully built gast
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.2 which is incompatible.
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2


pip list를 확인해 tensorflow 버전 확인

In [ ]:
%pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.10.0         
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.3.1          
astor                         0.8.1          
astropy                       4.1            
astunparse                    1.6.3          
async-generator               1.10           
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.3.0         
audioread                     2.1.9          
autograd                      1.3            
Babel                         2.9.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.2.1          
blis                          0.4.

만약 , tensorflow 버전이 바뀌지 않았다면

Ctrl+M 또는 상단 런타임/런타임 다시 시작

이후 드라이브 경로 이동 및 pip list 재실행, tensorflow 버전 1.15.0인지 확인

In [ ]:
%cd /content/drive/MyDrive/colab
%pip list

CVlineDetection.py 파일 실행

In [ ]:
%run CVlineDetection.py